In [1]:
import pandas as pd
import numpy as np

In [2]:
vehicles_df = pd.read_csv('data/vehicles.csv')
demand_df = pd.read_csv('data/demand.csv')

In [3]:
vehicles_df

,ID,Vehicle,Size,Year,Cost ($),Yearly range (km),Distance
0,BEV_S1_2023,BEV,S1,2023,187000,102000,D1
1,BEV_S1_2024,BEV,S1,2024,177650,102000,D1
2,BEV_S1_2025,BEV,S1,2025,168767,102000,D1
3,BEV_S1_2026,BEV,S1,2026,160329,102000,D2
4,BEV_S1_2027,BEV,S1,2027,152312,102000,D2
...,...,...,...,...,...,...,...
187,LNG_S3_2034,LNG,S3,2034,209208,73000,D4
188,LNG_S3_2035,LNG,S3,2035,215484,73000,D4
189,LNG_S3_2036,LNG,S3,2036,221948,73000,D4
190,LNG_S3_2037,LNG,S3,2037,228607,73000,D4


In [4]:
demand_df

,Year,Size,Distance,Demand (km)
0,2023,S1,D1,869181
1,2023,S1,D2,2597094
2,2023,S1,D3,3292011
3,2023,S1,D4,414315
4,2023,S2,D1,995694
...,...,...,...,...
251,2038,S4,D4,2446
252,2038,S3,D1,3229278
253,2038,S3,D2,3297618
254,2038,S3,D3,1448550


In [5]:
def allocate_vehicles(demand_df, vehicles_df, year):
    # Mapping for categorical distance values
    distance_mapping = {"D1": 1, "D2": 2, "D3": 3, "D4": 4}
    
    # Convert categorical distance to numeric ranks
    vehicles_df['Distance_categorical'] = vehicles_df['Distance'].map(distance_mapping)
    demand_df['Distance_categorical'] = demand_df['Distance'].map(distance_mapping)

    demand_df = demand_df[demand_df['Year'] == year].copy() # Filter relevant year
    available_vehicles = vehicles_df[vehicles_df['Year'] <= year]  # Vehicles available in that year

    def find_allocations(row):
        size_needed = row['Size']
        distance_needed = row['Distance_categorical']
        
        allocated_vehicles = available_vehicles[
            (available_vehicles['Size'] == size_needed) & 
            (available_vehicles['Distance_categorical'] >= distance_needed)
        ]['ID'].tolist() 
        
        return allocated_vehicles

    demand_df['Allocation'] = demand_df.apply(find_allocations, axis=1)  # Apply function row-wise
    
    return demand_df


In [6]:
allocation_df = allocate_vehicles(demand_df, vehicles_df, 2023)
allocation_df.to_csv('data/allocation_result.csv', index=False)

In [7]:
allocation_df

,Year,Size,Distance,Demand (km),Distance_categorical,Allocation
0,2023,S1,D1,869181,1,"[BEV_S1_2023, Diesel_S1_2023, LNG_S1_2023]"
1,2023,S1,D2,2597094,2,"[Diesel_S1_2023, LNG_S1_2023]"
2,2023,S1,D3,3292011,3,"[Diesel_S1_2023, LNG_S1_2023]"
3,2023,S1,D4,414315,4,"[Diesel_S1_2023, LNG_S1_2023]"
4,2023,S2,D1,995694,1,"[BEV_S2_2023, Diesel_S2_2023, LNG_S2_2023]"
5,2023,S2,D2,1383196,2,"[Diesel_S2_2023, LNG_S2_2023]"
6,2023,S2,D3,778008,3,"[Diesel_S2_2023, LNG_S2_2023]"
7,2023,S2,D4,133677,4,"[Diesel_S2_2023, LNG_S2_2023]"
8,2023,S4,D1,14576,1,"[BEV_S4_2023, Diesel_S4_2023, LNG_S4_2023]"
9,2023,S4,D2,754717,2,"[Diesel_S4_2023, LNG_S4_2023]"


In [8]:
df_exploded = allocation_df.explode("Allocation")

In [9]:
# Display the transformed DataFrame
print(df_exploded)

    Year Size Distance  Demand (km)  Distance_categorical      Allocation
0   2023   S1       D1       869181                     1     BEV_S1_2023
0   2023   S1       D1       869181                     1  Diesel_S1_2023
0   2023   S1       D1       869181                     1     LNG_S1_2023
1   2023   S1       D2      2597094                     2  Diesel_S1_2023
1   2023   S1       D2      2597094                     2     LNG_S1_2023
2   2023   S1       D3      3292011                     3  Diesel_S1_2023
2   2023   S1       D3      3292011                     3     LNG_S1_2023
3   2023   S1       D4       414315                     4  Diesel_S1_2023
3   2023   S1       D4       414315                     4     LNG_S1_2023
4   2023   S2       D1       995694                     1     BEV_S2_2023
4   2023   S2       D1       995694                     1  Diesel_S2_2023
4   2023   S2       D1       995694                     1     LNG_S2_2023
5   2023   S2       D2      1383196   

In [10]:
df = pd.merge(df_exploded[['Allocation', 'Year', 'Size', 'Distance', 'Demand (km)']], vehicles_df[['ID','Vehicle', 'Cost ($)', 'Yearly range (km)', 'Distance']], how='left', left_on='Allocation', right_on='ID')

In [11]:
df

,Allocation,Year,Size,Distance_x,Demand (km),ID,Vehicle,Cost ($),Yearly range (km),Distance_y
0,BEV_S1_2023,2023,S1,D1,869181,BEV_S1_2023,BEV,187000,102000,D1
1,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,85000,102000,D4
2,LNG_S1_2023,2023,S1,D1,869181,LNG_S1_2023,LNG,100000,102000,D4
3,Diesel_S1_2023,2023,S1,D2,2597094,Diesel_S1_2023,Diesel,85000,102000,D4
4,LNG_S1_2023,2023,S1,D2,2597094,LNG_S1_2023,LNG,100000,102000,D4
5,Diesel_S1_2023,2023,S1,D3,3292011,Diesel_S1_2023,Diesel,85000,102000,D4
6,LNG_S1_2023,2023,S1,D3,3292011,LNG_S1_2023,LNG,100000,102000,D4
7,Diesel_S1_2023,2023,S1,D4,414315,Diesel_S1_2023,Diesel,85000,102000,D4
8,LNG_S1_2023,2023,S1,D4,414315,LNG_S1_2023,LNG,100000,102000,D4
9,BEV_S2_2023,2023,S2,D1,995694,BEV_S2_2023,BEV,272000,106000,D1


In [12]:
# df['Num_Vehicles'] = np.ceil(df['Demand (km)']/df['Yearly range (km)'])

In [13]:
# df["Distance_per_vehicle(km)"] = df["Demand (km)"] / df["Num_Vehicles"]

In [12]:
df["Operating Year"] = 2023

In [13]:
df.columns

Index(['Allocation', 'Year', 'Size', 'Distance_x', 'Demand (km)', 'ID',
       'Vehicle', 'Cost ($)', 'Yearly range (km)', 'Distance_y',
       'Operating Year'],
      dtype='object')

In [14]:
vehicle_fuels_df = pd.read_csv('data/vehicles_fuels.csv')

In [15]:
df = pd.merge(df, vehicle_fuels_df, how='left', left_on='ID', right_on='ID')

In [16]:
df

,Allocation,Year,Size,Distance_x,Demand (km),ID,Vehicle,Cost ($),Yearly range (km),Distance_y,Operating Year,Fuel,Consumption (unit_fuel/km)
0,BEV_S1_2023,2023,S1,D1,869181,BEV_S1_2023,BEV,187000,102000,D1,2023,Electricity,0.893043
1,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,85000,102000,D4,2023,B20,0.223016
2,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,85000,102000,D4,2023,HVO,0.223016
3,LNG_S1_2023,2023,S1,D1,869181,LNG_S1_2023,LNG,100000,102000,D4,2023,LNG,0.165507
4,LNG_S1_2023,2023,S1,D1,869181,LNG_S1_2023,LNG,100000,102000,D4,2023,BioLNG,0.165159
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,LNG_S3_2023,2023,S3,D3,1002466,LNG_S3_2023,LNG,151136,73000,D4,2023,BioLNG,0.164684
64,Diesel_S3_2023,2023,S3,D4,205426,Diesel_S3_2023,Diesel,118181,73000,D4,2023,B20,0.224082
65,Diesel_S3_2023,2023,S3,D4,205426,Diesel_S3_2023,Diesel,118181,73000,D4,2023,HVO,0.224082
66,LNG_S3_2023,2023,S3,D4,205426,LNG_S3_2023,LNG,151136,73000,D4,2023,LNG,0.164823


In [17]:
df.drop('Consumption (unit_fuel/km)', axis=1, inplace=True)

In [18]:
df

,Allocation,Year,Size,Distance_x,Demand (km),ID,Vehicle,Cost ($),Yearly range (km),Distance_y,Operating Year,Fuel
0,BEV_S1_2023,2023,S1,D1,869181,BEV_S1_2023,BEV,187000,102000,D1,2023,Electricity
1,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,85000,102000,D4,2023,B20
2,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,85000,102000,D4,2023,HVO
3,LNG_S1_2023,2023,S1,D1,869181,LNG_S1_2023,LNG,100000,102000,D4,2023,LNG
4,LNG_S1_2023,2023,S1,D1,869181,LNG_S1_2023,LNG,100000,102000,D4,2023,BioLNG
...,...,...,...,...,...,...,...,...,...,...,...,...
63,LNG_S3_2023,2023,S3,D3,1002466,LNG_S3_2023,LNG,151136,73000,D4,2023,BioLNG
64,Diesel_S3_2023,2023,S3,D4,205426,Diesel_S3_2023,Diesel,118181,73000,D4,2023,B20
65,Diesel_S3_2023,2023,S3,D4,205426,Diesel_S3_2023,Diesel,118181,73000,D4,2023,HVO
66,LNG_S3_2023,2023,S3,D4,205426,LNG_S3_2023,LNG,151136,73000,D4,2023,LNG


In [19]:
from utilities.my_sql_operations import MySQLOperations
import pandas as pd

In [20]:
def max_carbon_emmissions_per_vehicle(fleet_details: pd.DataFrame, op_year: int):
        """
        Calculates total carbon emissions of the fleet
        """     
        my_sql_operations = MySQLOperations() 
        
        query = f"""SELECT fuel, emissions_co2_per_unit_fuel FROM fuels WHERE year = {op_year}"""
        fuel_data, columns = my_sql_operations.fetch_data(query) 
        fuel_df = pd.DataFrame(fuel_data, columns=columns)
        
        query = f"""SELECT * FROM vehicles_fuels"""
        vehicles_fuels_data, columns = my_sql_operations.fetch_data(query)
        vehicles_fuels_df = pd.DataFrame(vehicles_fuels_data, columns=columns)

        merged_df = pd.merge(
            pd.merge(fleet_details, fuel_df, left_on='Fuel', right_on='fuel', how='left'),
            vehicles_fuels_df, left_on=['ID', 'Fuel'], right_on=['id', 'fuel'], how='left'
        )
        
        merged_df['carbon_emissions'] = (
            merged_df['Yearly range (km)'] * 
            1 * # No. of vehicles
            merged_df['emissions_co2_per_unit_fuel'] *
            merged_df['consumption_unitfuel_per_km']
        )
        return merged_df['carbon_emissions']

In [21]:
df['carbon_emissions'] = max_carbon_emmissions_per_vehicle(df, 2023)

In [22]:
df

,Allocation,Year,Size,Distance_x,Demand (km),ID,Vehicle,Cost ($),Yearly range (km),Distance_y,Operating Year,Fuel,carbon_emissions
0,BEV_S1_2023,2023,S1,D1,869181,BEV_S1_2023,BEV,187000,102000,D1,2023,Electricity,0.000000
1,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,85000,102000,D4,2023,B20,69347.975963
2,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,85000,102000,D4,2023,HVO,11092.291306
3,LNG_S1_2023,2023,S1,D1,869181,LNG_S1_2023,LNG,100000,102000,D4,2023,LNG,41971.148530
4,LNG_S1_2023,2023,S1,D1,869181,LNG_S1_2023,LNG,100000,102000,D4,2023,BioLNG,6375.265894
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,LNG_S3_2023,2023,S3,D3,1002466,LNG_S3_2023,LNG,151136,73000,D4,2023,BioLNG,4549.567924
64,Diesel_S3_2023,2023,S3,D4,205426,Diesel_S3_2023,Diesel,118181,73000,D4,2023,B20,49868.628960
65,Diesel_S3_2023,2023,S3,D4,205426,Diesel_S3_2023,Diesel,118181,73000,D4,2023,HVO,7976.546565
66,LNG_S3_2023,2023,S3,D4,205426,LNG_S3_2023,LNG,151136,73000,D4,2023,LNG,29914.034489


In [23]:
def yearly_insurance_cost_per_vehicle(fleet_details: pd.DataFrame):
        """
        Returns Insurance cost for the operating year
        """
        total_fleet_insurance_cost = 0
        my_sql_operations = MySQLOperations() 
        
        query = f"""SELECT id, year, cost FROM vehicles"""
        purchase_year_data, columns = my_sql_operations.fetch_data(query) 
        purchase_year_df = pd.DataFrame(purchase_year_data, columns=columns)
        
        query = f"""SELECT end_of_year, insurance_cost_percent FROM cost_profiles"""
        insurance_cost_data, columns = my_sql_operations.fetch_data(query)
        insurance_cost_df = pd.DataFrame(insurance_cost_data, columns=columns)
        
        eoy_df = pd.merge(fleet_details, purchase_year_df, left_on=['ID'], right_on=['id'], how='left')
                
        eoy_df['End_of_year'] = (
            eoy_df['Operating Year'] - eoy_df['year'] + 1
        )
        
        merged_df = pd.merge(eoy_df, insurance_cost_df, left_on='End_of_year', right_on='end_of_year', how='left')
        
        merged_df['insurance_cost'] = (
            ((merged_df['insurance_cost_percent']/100) * merged_df['cost']) * 1 #No of vehicles 
        )
        return merged_df['insurance_cost']
            
def yearly_maintenance_cost_per_vehicle(fleet_details: pd.DataFrame):
        """
        Returns Maintenance cost for the operating year
        """
        total_fleet_maintainance_cost = 0
        my_sql_operations = MySQLOperations() 
        
        query = f"""SELECT id, year, cost FROM vehicles"""
        purchase_year_data, columns = my_sql_operations.fetch_data(query) 
        purchase_year_df = pd.DataFrame(purchase_year_data, columns=columns)
        
        query = f"""SELECT end_of_year, maintenance_cost_percent FROM cost_profiles"""
        maintenance_cost_data, columns = my_sql_operations.fetch_data(query)
        maintenance_cost_df = pd.DataFrame(maintenance_cost_data, columns=columns)
        
        eoy_df = pd.merge(fleet_details, purchase_year_df, left_on=['ID'], right_on=['id'], how='left')
                
        eoy_df['End_of_year'] = (
            eoy_df['Operating Year'] - eoy_df['year'] + 1
        )
        
        merged_df = pd.merge(eoy_df, maintenance_cost_df, left_on='End_of_year', right_on='end_of_year', how='left')
        
        merged_df['maintenance_cost'] = (
            ((merged_df['maintenance_cost_percent']/100) * merged_df['cost']) * 1 #No of vehicles 
        )
        return merged_df['maintenance_cost']
          
def yearly_fuel_cost_per_vehicle(fleet_details: pd.DataFrame, op_year: int):
        """
        Returns yearly fuel cost
        """
        yearly_fuel_cost = 0
        my_sql_operations = MySQLOperations() 
        
        query = f"""SELECT fuel, cost_per_unit_fuel FROM fuels WHERE year = {op_year}"""
        fuel_cost_data, columns = my_sql_operations.fetch_data(query) 
        fuel_cost_df = pd.DataFrame(fuel_cost_data, columns=columns)
        
        query = f"""SELECT * FROM vehicles_fuels"""
        fuel_consumption_data, columns = my_sql_operations.fetch_data(query)
        fuel_consumption_df = pd.DataFrame(fuel_consumption_data, columns=columns)
        
        merged_df = pd.merge(
            pd.merge(fleet_details, fuel_cost_df, left_on='Fuel', right_on='fuel', how='left'),
            fuel_consumption_df, left_on=['ID', 'Fuel'], right_on=['id', 'fuel'], how='left'
        )
        
        merged_df['fuel_costs'] = (
            merged_df['Yearly range (km)'] * 
            1 * # No of vehicles 
            merged_df['consumption_unitfuel_per_km'] *
            merged_df['cost_per_unit_fuel']
        )
        return merged_df['fuel_costs']

In [24]:
df['insurance_cost'] = yearly_insurance_cost_per_vehicle(df)
df['maintenance_cost'] = yearly_maintenance_cost_per_vehicle(df)
df['fuel_costs'] = yearly_fuel_cost_per_vehicle(df, 2023)


In [25]:
df

,Allocation,Year,Size,Distance_x,Demand (km),ID,Vehicle,Cost ($),Yearly range (km),Distance_y,Operating Year,Fuel,carbon_emissions,insurance_cost,maintenance_cost,fuel_costs
0,BEV_S1_2023,2023,S1,D1,869181,BEV_S1_2023,BEV,187000,102000,D1,2023,Electricity,0.000000,9350.00,1870.00,17470.316221
1,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,85000,102000,D4,2023,B20,69347.975963,4250.00,850.00,27771.219051
2,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,85000,102000,D4,2023,HVO,11092.291306,4250.00,850.00,41271.256214
3,LNG_S1_2023,2023,S1,D1,869181,LNG_S1_2023,LNG,100000,102000,D4,2023,LNG,41971.148530,5000.00,1000.00,17077.879517
4,LNG_S1_2023,2023,S1,D1,869181,LNG_S1_2023,LNG,100000,102000,D4,2023,BioLNG,6375.265894,5000.00,1000.00,18325.652865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,LNG_S3_2023,2023,S3,D3,1002466,LNG_S3_2023,LNG,151136,73000,D4,2023,BioLNG,4549.567924,7556.80,1511.36,13077.698068
64,Diesel_S3_2023,2023,S3,D4,205426,Diesel_S3_2023,Diesel,118181,73000,D4,2023,B20,49868.628960,5909.05,1181.81,19970.483628
65,Diesel_S3_2023,2023,S3,D4,205426,Diesel_S3_2023,Diesel,118181,73000,D4,2023,HVO,7976.546565,5909.05,1181.81,29678.457580
66,LNG_S3_2023,2023,S3,D4,205426,LNG_S3_2023,LNG,151136,73000,D4,2023,LNG,29914.034489,7556.80,1511.36,12171.891758


In [26]:
df.to_csv('data/something.csv')

In [ ]:
transaction_types = ["Buy", "Sell", "Use"]

In [28]:
df["Type"] = [transaction_types for i in df.index]

In [29]:
df_exploded = df.explode("Type")

In [30]:
df_exploded

,Allocation,Year,Size,Distance_x,Demand (km),ID,Vehicle,Cost ($),Yearly range (km),Distance_y,Operating Year,Fuel,carbon_emissions,insurance_cost,maintenance_cost,fuel_costs,Type
0,BEV_S1_2023,2023,S1,D1,869181,BEV_S1_2023,BEV,187000,102000,D1,2023,Electricity,0.000000,9350.0,1870.00,17470.316221,Buy
0,BEV_S1_2023,2023,S1,D1,869181,BEV_S1_2023,BEV,187000,102000,D1,2023,Electricity,0.000000,9350.0,1870.00,17470.316221,Sell
0,BEV_S1_2023,2023,S1,D1,869181,BEV_S1_2023,BEV,187000,102000,D1,2023,Electricity,0.000000,9350.0,1870.00,17470.316221,Use
1,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,85000,102000,D4,2023,B20,69347.975963,4250.0,850.00,27771.219051,Buy
1,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,85000,102000,D4,2023,B20,69347.975963,4250.0,850.00,27771.219051,Sell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,LNG_S3_2023,2023,S3,D4,205426,LNG_S3_2023,LNG,151136,73000,D4,2023,LNG,29914.034489,7556.8,1511.36,12171.891758,Sell
66,LNG_S3_2023,2023,S3,D4,205426,LNG_S3_2023,LNG,151136,73000,D4,2023,LNG,29914.034489,7556.8,1511.36,12171.891758,Use
67,LNG_S3_2023,2023,S3,D4,205426,LNG_S3_2023,LNG,151136,73000,D4,2023,BioLNG,4549.567924,7556.8,1511.36,13077.698068,Buy
67,LNG_S3_2023,2023,S3,D4,205426,LNG_S3_2023,LNG,151136,73000,D4,2023,BioLNG,4549.567924,7556.8,1511.36,13077.698068,Sell


In [33]:
def buy_costs(row):
    return row["Cost ($)"]
def use_costs():
    return None # it should return 0 if vehicle is present in previous years fleet
def resale_value(row):
    my_sql_operations = MySQLOperations() 
        
    query = f"""SELECT end_of_year, resale_value_percent FROM cost_profiles"""
    resale_value_data, columns = my_sql_operations.fetch_data(query)
    resale_value_df = pd.DataFrame(resale_value_data, columns=columns)

    eoy= row['Operating Year'] - row['Year'] + 1
    for index, _ in resale_value_df.iterrows():
        if _['end_of_year'] == eoy:
            resale_value = (_['resale_value_percent']/100) * row['Cost ($)']
    return resale_value

In [34]:
def calculate_cost(row):
    if row["Type"] == "Buy":
        return buy_costs(row)
    elif row["Type"] == "Use":
        return use_costs()
    elif row["Type"] == "Sell":
        return resale_value(row)

In [35]:
df_exploded["Cost"] = df_exploded.apply(calculate_cost, axis=1)

In [36]:
df_exploded.reset_index(drop=True, inplace=True)

In [37]:
df_exploded

,Allocation,Year,Size,Distance_x,Demand (km),ID,Vehicle,Cost ($),Yearly range (km),Distance_y,Operating Year,Fuel,carbon_emissions,insurance_cost,maintenance_cost,fuel_costs,Type,Cost
0,BEV_S1_2023,2023,S1,D1,869181,BEV_S1_2023,BEV,187000,102000,D1,2023,Electricity,0.000000,9350.0,1870.00,17470.316221,Buy,187000.0
1,BEV_S1_2023,2023,S1,D1,869181,BEV_S1_2023,BEV,187000,102000,D1,2023,Electricity,0.000000,9350.0,1870.00,17470.316221,Sell,168300.0
2,BEV_S1_2023,2023,S1,D1,869181,BEV_S1_2023,BEV,187000,102000,D1,2023,Electricity,0.000000,9350.0,1870.00,17470.316221,Use,NaN
3,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,85000,102000,D4,2023,B20,69347.975963,4250.0,850.00,27771.219051,Buy,85000.0
4,Diesel_S1_2023,2023,S1,D1,869181,Diesel_S1_2023,Diesel,85000,102000,D4,2023,B20,69347.975963,4250.0,850.00,27771.219051,Sell,76500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,LNG_S3_2023,2023,S3,D4,205426,LNG_S3_2023,LNG,151136,73000,D4,2023,LNG,29914.034489,7556.8,1511.36,12171.891758,Sell,136022.4
200,LNG_S3_2023,2023,S3,D4,205426,LNG_S3_2023,LNG,151136,73000,D4,2023,LNG,29914.034489,7556.8,1511.36,12171.891758,Use,NaN
201,LNG_S3_2023,2023,S3,D4,205426,LNG_S3_2023,LNG,151136,73000,D4,2023,BioLNG,4549.567924,7556.8,1511.36,13077.698068,Buy,151136.0
202,LNG_S3_2023,2023,S3,D4,205426,LNG_S3_2023,LNG,151136,73000,D4,2023,BioLNG,4549.567924,7556.8,1511.36,13077.698068,Sell,136022.4


In [38]:
df_exploded.to_csv('data/something_new.csv', index=False)

In [40]:
df = df_exploded.dropna(subset = ['Cost'])

In [42]:
df.to_csv('data/something_new.csv', index=False)

In [44]:
combinations = df.groupby(["Size", "Distance_x"])

In [48]:
for (size, distance_x), group in combinations:
    

S1 D1
        Allocation  Year Size Distance_x  Demand (km)              ID Vehicle  \
0      BEV_S1_2023  2023   S1         D1       869181     BEV_S1_2023     BEV   
1      BEV_S1_2023  2023   S1         D1       869181     BEV_S1_2023     BEV   
3   Diesel_S1_2023  2023   S1         D1       869181  Diesel_S1_2023  Diesel   
4   Diesel_S1_2023  2023   S1         D1       869181  Diesel_S1_2023  Diesel   
6   Diesel_S1_2023  2023   S1         D1       869181  Diesel_S1_2023  Diesel   
7   Diesel_S1_2023  2023   S1         D1       869181  Diesel_S1_2023  Diesel   
9      LNG_S1_2023  2023   S1         D1       869181     LNG_S1_2023     LNG   
10     LNG_S1_2023  2023   S1         D1       869181     LNG_S1_2023     LNG   
12     LNG_S1_2023  2023   S1         D1       869181     LNG_S1_2023     LNG   
13     LNG_S1_2023  2023   S1         D1       869181     LNG_S1_2023     LNG   

    Cost ($)  Yearly range (km) Distance_y  Operating Year         Fuel  \
0     187000             10